In [1]:
from ultralytics import YOLO
import cv2
import pandas as pd
import os

In [2]:
# Charger le modèle YOLOv8 (vous pouvez utiliser un modèle personnalisé ou un modèle pré-entraîné)
model = YOLO("yolov8n.pt")  # Utilisez 'yolov8n.pt' pour la version nano

In [9]:
def convert_video_with_opencv(input_path, output_path):
    # Ouvrir la vidéo en entrée
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Erreur : Impossible d'ouvrir la vidéo.")
        return

    # Récupérer les propriétés de la vidéo
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec pour .mp4

    # Définir la vidéo en sortie
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))


    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Écrire chaque frame dans le fichier de sortie
        out.write(frame)

    # Libérer les ressources
    cap.release()
    out.release()
    print(f"Vidéo convertie avec succès : {output_path}")

# Chemin de la vidéo .mov dans l'environnement Kaggle
input_video = "../Videos MOV/video10.mov"
# Chemin pour la vidéo convertie dans l'environnement Kaggle (répertoire de travail)
output_video = "../Data/Videos/video10.mp4"

convert_video_with_opencv(input_video, output_video)

Vidéo convertie avec succès : C:/Users/h/Desktop/MASTER IAAD/S3/projet2/Data/Videos/video10.mp4


In [93]:
# Chemin vers la vidéo ou le dossier d'images
input_path = "../Data/Videos/video12.mp4" # Remplacez par le chemin de votre vidéo ou dossier d'images
output_path = "../Videos Annoter/video12.mp4"  # Chemin pour sauvegarder la vidéo annotée
csv_path = "../Data/annotations/video12.csv"  # Chemin pour sauvegarder les résultats en CSV

In [94]:
# Ouvrir la vidéo ou le dossier d'images
# Initialize frame_id for both video and images
frame_id = 0
if input_path.endswith(".mp4"):
    cap = cv2.VideoCapture(input_path)
else:
    # Si c'est un dossier d'images, lisez-les séquentiellement
    image_files = sorted([os.path.join(input_path, f) for f in os.listdir(input_path) if f.endswith(('.jpg', '.png'))])
    frame_id = 0

In [95]:
# Obtenir les propriétés de la vidéo (largeur, hauteur, fps)
if input_path.endswith(".mp4"):
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
else:
    # Utiliser les dimensions de la première image pour le dossier d'images
    first_image = cv2.imread(image_files[0])
    frame_height, frame_width, _ = first_image.shape
    fps = 30  # Définir un fps par défaut pour les images

In [96]:
# Définir le codec et créer un objet VideoWriter pour sauvegarder la vidéo
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec pour .mp4
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

In [97]:
# Liste pour stocker les résultats
results_list = []

In [98]:
while True:
    if input_path.endswith(".mp4"):
        ret, frame = cap.read()
        if not ret:
            break  # Sortir de la boucle si la vidéo est terminée
    else:
        if frame_id >= len(image_files):
            break  # Sortir de la boucle si toutes les images sont traitées
        frame = cv2.imread(image_files[frame_id])
        frame_id += 1

    # Appliquer YOLOv8 avec suivi
    results = model.track(frame, persist=True, tracker="bytetrack.yaml")  # Activer le suivi

    # Annoter la frame avec les résultats
    annotated_frame = results[0].plot()  # Dessiner les boîtes englobantes et les IDs

    # Écrire la frame annotée dans la vidéo de sortie
    out.write(annotated_frame)

    # Extraire les informations des objets détectés
    for obj in results[0].boxes:
        object_id = int(obj.id) if obj.id is not None else -1  # ID de l'objet (si suivi activé)
        object_type = model.names[int(obj.cls)]  # Type de l'objet (classe)
        bbox = obj.xywh[0].tolist()  # Boîte englobante (x_centre, y_centre, largeur, hauteur)
        position_x, position_y, object_width, object_height = bbox

        # Calculer la position Z (si disponible, sinon mettre à 0)
        position_z = 0  # À adapter si vous avez des informations sur la profondeur

        # Calculer l'orientation (heading) (si disponible, sinon mettre à 0)
        heading = 0  # À adapter si vous avez des informations sur l'orientation

        # Ajouter les informations à la liste des résultats
        results_list.append([
            frame_id, object_id, object_type, position_x, position_y, position_z,object_height, object_width, object_height, heading
        ])

    # (Optionnel) Afficher la frame annotée en temps réel
    cv2.imshow("Tracking", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Appuyer sur 'q' pour quitter
        break

    frame_id += 1


0: 384x640 1 bus, 200.6ms
Speed: 4.0ms preprocess, 200.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 157.4ms
Speed: 4.0ms preprocess, 157.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 173.7ms
Speed: 6.0ms preprocess, 173.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 129.4ms
Speed: 5.0ms preprocess, 129.4ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 131.5ms
Speed: 6.0ms preprocess, 131.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 134.7ms
Speed: 4.0ms preprocess, 134.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 124.7ms
Speed: 6.0ms preprocess, 124.7ms inference, 3.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 243.4ms
Speed: 6.0ms preprocess, 243.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 

In [99]:
# Libérer les ressources
if input_path.endswith(".mp4"):
    cap.release()
out.release()
cv2.destroyAllWindows()

In [100]:
# Convertir la liste des résultats en DataFrame
columns = [
    "frame_id", "object_id", "object_type", "position_x", "position_y", "position_z",
    "object_length", "object_width", "object_height", "heading"
]
results_df = pd.DataFrame(results_list, columns=columns)

# Sauvegarder les résultats dans un fichier CSV
results_df.to_csv(csv_path, index=False)

print(f"Vidéo annotée sauvegardée sous : {output_path}")
print(f"Résultats de détection sauvegardés sous : {csv_path}")

Vidéo annotée sauvegardée sous : ../Videos Annoter/video12.mp4
Résultats de détection sauvegardés sous : ../Data/annotations/video12.csv
